## Prepare data reader and model execution context

In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.modelspec import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "account_abstraction"


# Select a model.
data_spec = ModelsDataSpec(root_path_prefix="blockbatch", models=[model_name])

# Select a block batch.
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="25821200:+1",
    root_paths_to_read=data_spec.input_root_paths,
)

# Construct readers
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)

# Show details for the batch we are processing.
pprint(readers[0])

# Ensure existence of data needed by the reader.
assert readers[0].inputs_ready

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-03-03 21:28:46 [debug    ] connecting to GOLDSKY Clickhouse client... filename=client.py lineno=56 process=93694
2025-03-03 21:28:46 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=93694
2025-03-03 21:28:46 [debug    ] loaded vault: 19 items         filename=vault.py lineno=79 process=93694
2025-03-03 21:28:47 [debug    ] initialized GOLDSKY Clickhouse client. filename=client.py lineno=61 process=93694
2025-03-03 21:28:47 [info     ] querying markers at time range min: 2025-02-01 17:42:27, max: 2025-02-01 17:42:27 root_paths=['blockbatch/account_abstraction_prefilter/entrypoint_logs_v1', 'blockbatch/account_abstraction_prefilter/entrypoint_traces_v1'] filename=request.py lineno=146 process=93694
2025-03-03 21:28:47 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=56 process=93694
2025-03-03 21:28:47 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=61 process=93694
2025-03-03 21:28:47 [info    

In [2]:
from op_analytics.datapipeline.models.code.account_abstraction.decoders import (
    register_4337_decoders,
)

from op_analytics.datapipeline.models.code.account_abstraction.abis import (
    HANDLE_OPS_FUNCTION_METHOD_ID_v0_6_0,
    HANDLE_OPS_FUNCTION_METHOD_ID_v0_7_0,
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
)

register_4337_decoders(ctx)


# Decoded UserOperationEvent logs.
user_ops = auxiliary_templates["account_abstraction/useroperationevent_logs"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": input_datasets[
            "blockbatch/account_abstraction_prefilter/entrypoint_logs_v1"
        ].as_subquery(),
    },
)

prefiltered_traces = input_datasets["blockbatch/account_abstraction_prefilter/entrypoint_traces_v1"].create_table(
    additional_sql="ORDER BY block_number, transaction_hash",
)

# Traces initiated on behalf of the UserOperationEvent sender
entrypoint_traces = auxiliary_templates[
    "account_abstraction/enriched_entrypoint_traces"
].create_table(
    duckdb_context=ctx,
    template_parameters={
        "prefiltered_traces": prefiltered_traces,
        "uops": user_ops,
        "method_id_v6": INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
        "method_id_v7": INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
        "handle_ops_v6": HANDLE_OPS_FUNCTION_METHOD_ID_v0_6_0,
        "handle_ops_v7": HANDLE_OPS_FUNCTION_METHOD_ID_v0_7_0,
    },
)

# Data Quality Checks
errors = []
for name, val in auxiliary_templates.items():
    if "data_quality_check" in name:
        errors.extend(val.run_as_data_quality_check(duckdb_context=ctx))
if errors:
    raise Exception("\n\n".join([name] + [str(_) for _ in errors]))
else:
    print("Data Quality OK")


2025-03-03 21:28:47 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=283 process=93694
2025-03-03 21:28:47 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=93694 template=account_abstraction/useroperationevent_logs
2025-03-03 21:28:49 [info     ] duck db size: 12.3KB           filename=client.py lineno=40 process=93694
2025-03-03 21:28:49 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=283 process=93694
2025-03-03 21:28:50 [info     ] created table/view blockbatch_account_abstraction_prefilter_entrypoint_traces_v1_tbl filename=client.py lineno=222 process=93694
2025-03-03 21:28:50 [info     ] duck db size: 12.3KB           filename=client.py lineno=40 process=93694
2025-03-03 21:28:50 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=93694 template=account_abstraction/enriched_entrypoint_traces
2025-03-03 21:28:50 [info     ] duck db size: 12

In [3]:
ctx.client.sql("SHOW TABLES")

┌───────────────────────────────────────────────────────────────────┐
│                               name                                │
│                              varchar                              │
├───────────────────────────────────────────────────────────────────┤
│ account_abstraction__enriched_entrypoint_traces                   │
│ account_abstraction__useroperationevent_logs                      │
│ blockbatch_account_abstraction_prefilter_entrypoint_traces_v1_tbl │
└───────────────────────────────────────────────────────────────────┘

In [4]:
ctx.client.sql(f"""
    SELECT 
        transaction_hash, 
        trace_root, 
        trace_address, 
        innerhandleop_trace_address, 
        is_from_sender, 
        is_innerhandleop,
        userop_sender,
        useropevent_success,
        status,
        error,
        from_address, 
        to_address
    FROM {entrypoint_traces} 
    WHERE
        useropevent_success 
        AND status = 0
""").show(max_rows=100)

┌────────────────────────────────────────────────────────────────────┬────────────┬───────────────┬─────────────────────────────┬────────────────┬──────────────────┬────────────────────────────────────────────┬─────────────────────┬────────┬────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┐
│                          transaction_hash                          │ trace_root │ trace_address │ innerhandleop_trace_address │ is_from_sender │ is_innerhandleop │               userop_sender                │ useropevent_success │ status │       error        │                from_address                │                 to_address                 │
│                              varchar                               │   int32    │    varchar    │           varchar           │    boolean     │     boolean      │                  varchar                   │       boolean       │ int64  │      varchar       │                  varchar       

In [15]:
# SUCCESSFUL USEROP WITH A REVERTED TRACE

ctx.client.sql(f"""
    SELECT 
        transaction_hash, 
        tx_from_address,
        bundler_address,
        trace_root, 
        trace_address, 
        innerhandleop_trace_address, 
        is_from_sender, 
        is_innerhandleop,
        useropevent_success,
        status,
        error,
        userop_sender,
        from_address, 
        to_address,
        userop_idx,
        useropevent_actualgascost_eth
    FROM {entrypoint_traces} 
    WHERE transaction_hash = '0xcb761ed55915a26aeb995a05c91a00974afeb8fa4bec142345876ede847a4574'
    ORDER BY TRY_CAST(trace_root AS INT), trace_address
""").show(max_rows=100)

┌────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────┬───────────────┬─────────────────────────────┬────────────────┬──────────────────┬─────────────────────┬────────┬────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────┬───────────────────────────────┐
│                          transaction_hash                          │              tx_from_address               │              bundler_address               │ trace_root │ trace_address │ innerhandleop_trace_address │ is_from_sender │ is_innerhandleop │ useropevent_success │ status │       error        │               userop_sender                │                from_address                │                 to_address                 │ userop_idx │ useropevent_actualgascost_eth │
│                       

In [13]:
# STANDARD AA BUNDLE


ctx.client.sql(f"""
    SELECT 
        transaction_hash, 
        tx_from_address,
        bundler_address,
        trace_root, 
        trace_address, 
        innerhandleop_trace_address, 
        is_from_sender, 
        is_innerhandleop,
        useropevent_success,
        status,
        error,
        userop_sender,
        from_address, 
        to_address,
        userop_idx,
        useropevent_actualgascost_eth
    FROM {entrypoint_traces} 
    WHERE transaction_hash = '0x85dc9e8463b762edcd134a885a8575ed5c6ab0484223ff33856bb3fec838c552'
    ORDER BY TRY_CAST(trace_root AS INT), trace_address
""").show(max_rows=100)

┌────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────┬───────────────┬─────────────────────────────┬────────────────┬──────────────────┬─────────────────────┬────────┬─────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────┬───────────────────────────────┐
│                          transaction_hash                          │              tx_from_address               │              bundler_address               │ trace_root │ trace_address │ innerhandleop_trace_address │ is_from_sender │ is_innerhandleop │ useropevent_success │ status │  error  │               userop_sender                │                from_address                │                 to_address                 │ userop_idx │ useropevent_actualgascost_eth │
│                              varchar        

In [7]:
## NON-STANDARD AA BUNDLE

ctx.client.sql(f"""
    SELECT 
        transaction_hash, 
        trace_root, 
        trace_address, 
        innerhandleop_trace_address, 
        is_from_sender, 
        is_innerhandleop,
        useropevent_success,
        status,
        error,
        userop_sender, 
        from_address, 
        to_address
    FROM {entrypoint_traces} 
    WHERE innerhandleop_trace_address.contains(',')
    ORDER BY TRY_CAST(trace_root AS INT), trace_address
""").show(max_rows=100)

┌──────────────────┬────────────┬───────────────┬─────────────────────────────┬────────────────┬──────────────────┬─────────────────────┬────────┬─────────┬───────────────┬──────────────┬────────────┐
│ transaction_hash │ trace_root │ trace_address │ innerhandleop_trace_address │ is_from_sender │ is_innerhandleop │ useropevent_success │ status │  error  │ userop_sender │ from_address │ to_address │
│     varchar      │   int32    │    varchar    │           varchar           │    boolean     │     boolean      │       boolean       │ int64  │ varchar │    varchar    │   varchar    │  varchar   │
├──────────────────┴────────────┴───────────────┴─────────────────────────────┴────────────────┴──────────────────┴─────────────────────┴────────┴─────────┴───────────────┴──────────────┴────────────┤
│                                                                                                0 rows                                                                                             

In [8]:
import polars as pl

rel = ctx.client.sql(f"""
    SELECT * FROM {user_ops} 
    WHERE transaction_hash = '0x85dc9e8463b762edcd134a885a8575ed5c6ab0484223ff33856bb3fec838c552'
    ORDER BY log_index
    """)

with pl.Config(
    tbl_cols=-1,
    tbl_rows=-1,
    tbl_width_chars=1200,
    fmt_str_lengths=128,
    tbl_formatting="MARKDOWN",
    tbl_hide_column_data_types=False,
    tbl_hide_dataframe_shape=True,
) as cfg:
    print(rel.pl())


rel


| dt         | chain | chain_id | network | block_timestamp | block_number | block_hash                                                         | transaction_hash                                                   | transaction_index | log_index | contract_address                           | userophash                                                         | sender                                     | paymaster                                  | nonce | success | actualGasCost | actualGasUsed |
| ---        | ---   | ---      | ---     | ---             | ---          | ---                                                                | ---                                                                | ---               | ---       | ---                                        | ---                                                                | ---                                        | ---                                        | ---   | ---     | ---           | ---           

┌────────────┬─────────┬──────────┬─────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬───────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────┬─────────┬───────────────┬───────────────┐
│     dt     │  chain  │ chain_id │ network │ block_timestamp │ block_number │                             block_hash                             │                          transaction_hash                          │ transaction_index │ log_index │              contract_address              │                             userophash                             │                   sender                   │                 paymaster                  │  nonce  │ success │ actualGasCost │ actual

In [9]:
with pl.Config(
    tbl_cols=-1,
    tbl_rows=-1,
    tbl_width_chars=1200,
    fmt_str_lengths=128,
    tbl_formatting="MARKDOWN",
    tbl_hide_column_data_types=True,
    tbl_hide_dataframe_shape=True,
) as cfg:
    df1 = ctx.client.sql(f"DESCRIBE TABLE {user_ops}").select("column_name", "column_type").pl()
    print(df1)
    print()
    
    df2 = ctx.client.sql(f"DESCRIBE TABLE {entrypoint_traces}").select("column_name", "column_type").pl()
    print(df2)
    print()

| column_name       | column_type |
|-------------------|-------------|
| dt                | DATE        |
| chain             | VARCHAR     |
| chain_id          | INTEGER     |
| network           | VARCHAR     |
| block_timestamp   | UINTEGER    |
| block_number      | BIGINT      |
| block_hash        | VARCHAR     |
| transaction_hash  | VARCHAR     |
| transaction_index | BIGINT      |
| log_index         | BIGINT      |
| contract_address  | VARCHAR     |
| userophash        | VARCHAR     |
| sender            | VARCHAR     |
| paymaster         | VARCHAR     |
| nonce             | VARCHAR     |
| success           | BOOLEAN     |
| actualGasCost     | VARCHAR     |
| actualGasUsed     | VARCHAR     |

| column_name                   | column_type |
|-------------------------------|-------------|
| dt                            | DATE        |
| chain                         | VARCHAR     |
| chain_id                      | INTEGER     |
| network                       | VARCH

In [11]:
ctx.client.sql(f"""
    SELECT 
        transaction_hash, 
        tx_from_address,
        bundler_address,
        trace_root, 
        trace_address, 
        innerhandleop_trace_address, 
        is_from_sender, 
        is_innerhandleop,
        useropevent_success,
        status,
        error,
        userop_sender,
        from_address, 
        to_address
    FROM {entrypoint_traces} 
    
    ORDER BY  transaction_hash, TRY_CAST(trace_root AS INT), trace_address
    LIMIT 40
""").show(max_rows=100)

┌────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────┬──────────────────────┬─────────────────────────────┬────────────────┬──────────────────┬─────────────────────┬────────┬─────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┐
│                          transaction_hash                          │              tx_from_address               │              bundler_address               │ trace_root │    trace_address     │ innerhandleop_trace_address │ is_from_sender │ is_innerhandleop │ useropevent_success │ status │  error  │               userop_sender                │                from_address                │                 to_address                 │
│                              varchar                               │                  varchar                   │       